In [1]:
print("""
@Title        : Time Series Analysis and Forecasting
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2023-12-24 00:39:34
@Description  : 
""")


@Title        : Time Series Analysis and Forecasting
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2023-12-24 00:39:34
@Description  : 



## Time series decomposition

In [3]:
import pandas as pd
import quandl
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
with open('../../token/quandl.txt', 'r') as f:
    token = f.read()
quandl.ApiConfig.api_key = token

In [25]:
df = (
    pd.read_csv('../data/UNRATENSA.csv', parse_dates=['DATE'], index_col='DATE')
    .rename_axis('Date', axis=0)
    .rename(columns={'UNRATENSA': 'unemp_rate'}))
df.sample(5)

,unemp_rate
Date,
2010-08-01,9.5
2010-11-01,9.3
2016-11-01,4.4
2017-10-01,3.9
2013-10-01,7.0


In [27]:
temp_df = df.copy()
temp_df['year'] = temp_df.index.year